In [1]:
import os
import json
from pathlib import Path
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt


from data import get_dataset, get_zoo_elephants_images_and_labels, get_ELEP_images_and_labels, parse_image_function
from train import SiameseModel
import triplet_loss


import tensorflow as tf
from tensorflow.keras import optimizers

from tensorflow.keras.applications.resnet_v2 import preprocess_input

In [19]:
from importlib import reload
reload(triplet_loss)

<module 'triplet_loss' from '/Users/deepakduggirala/Documents/project/siamese/triplet_loss.py'>

In [2]:
with open('hyperparameters/initial_run.json', 'rb') as f:
    params = json.load(f)

In [3]:
params['batch_size']['train'] = 32
params['batch_size']['val'] = 512

In [4]:
data_dir = '/Users/deepakduggirala/Documents/ELPephant-cropped'

In [5]:
cache_files = {
        'train': str(Path(data_dir) / 'train.cache'),
        'val': str(Path(data_dir) / 'val.cache')
    }

In [6]:
train_ds, N_train = get_dataset(get_ELEP_images_and_labels, params, data_dir, 'val', 
                                augment=True,
                                cache_files=cache_files, 
                                model_preprocess=False, 
                                shuffle=True)
# train_ds_aug, N_train = get_dataset(get_ELEP_images_and_labels, params, data_dir, 'val', 
#                                 augment=True,
#                                 cache_files=None, 
#                                 model_preprocess=False, 
#                                 shuffle=False)

In [7]:
images, labels = next(train_ds.as_numpy_iterator())
# images_aug, labels = next(train_ds_aug.as_numpy_iterator())

2022-04-26 13:42:53.699857: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
siamese_model = SiameseModel(params, False)

In [9]:
siamese_model.compile(optimizer=optimizers.Adam(learning_rate=params['lr']))
input_shape = (None, params['image_size'], params['image_size'], 3)
siamese_model.compute_output_shape(input_shape=input_shape)

TensorShape([None, 256])

In [10]:
emb = siamese_model.predict(images)

In [11]:
emb = emb / np.linalg.norm(emb, axis=1, keepdims=1)

In [12]:
tf_emb = tf.convert_to_tensor(emb.astype(np.float32))
tf_labels = tf.convert_to_tensor(labels)

In [21]:
pairwise_dist = triplet_loss._pairwise_distances(tf_emb, squared=False)

In [22]:
mask_anchor_positive = triplet_loss._get_anchor_positive_triplet_mask(labels)
mask_anchor_positive = tf.cast(mask_anchor_positive, dtype=tf.float32)

anchor_positive_dist = tf.multiply(mask_anchor_positive, pairwise_dist)

In [23]:
hardest_positive_dist = tf.reduce_max(anchor_positive_dist, axis=1, keepdims=True)

In [24]:
hard_positive_indices = tf.math.argmax(anchor_positive_dist, axis=1)

In [32]:
mask_anchor_negative = triplet_loss._get_anchor_negative_triplet_mask(labels)
mask_anchor_negative = tf.cast(mask_anchor_negative, dtype=tf.float32)

In [33]:
max_anchor_negative_dist = tf.reduce_max(pairwise_dist, axis=1, keepdims=True)
anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)
hardest_negative_dist = tf.reduce_min(anchor_negative_dist, axis=1, keepdims=True)

In [34]:
triplet_loss = tf.maximum(hardest_positive_dist - hardest_negative_dist + 0.5, 0.0)

In [93]:
tf.reduce_sum(triplet_loss)

<tf.Tensor: shape=(), dtype=float32, numpy=339.20908>

In [101]:
anchor_positive_dist

<tf.Tensor: shape=(457, 457), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.24179254, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.24179254, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [112]:
tf.reduce_sum(mask_anchor_positive)

<tf.Tensor: shape=(), dtype=float32, numpy=4690.0>

In [ ]:
anchor_positive_dist - (hardest_negative_dist * mask_anchor_positive)

In [115]:
tf.reduce_sum(tf.maximum(anchor_positive_dist - (hardest_negative_dist * mask_anchor_positive) + 0.5 * mask_anchor_positive, 0.0))

<tf.Tensor: shape=(), dtype=float32, numpy=2966.9807>

In [16]:
triplet_loss.batch_hard_triplet_loss(tf_labels, tf_emb, margin=0.5, squared=False)

<tf.Tensor: shape=(), dtype=float32, numpy=0.74140745>

In [18]:
triplet_loss.batch_hard_triplet_loss(tf_labels, tf_emb, margin=0.5, squared=False)

<tf.Tensor: shape=(), dtype=float32, numpy=0.07224376>

In [20]:
triplet_loss.batch_partial_hard_triplet_loss(tf_labels, tf_emb, margin=0.5, squared=False)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6342425>

In [31]:
np.argmin(np.sum(mask_anchor_positive.numpy(),axis=1))

172

In [36]:
triplet_loss[172]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.40069026], dtype=float32)>